In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
from fairlearn.metrics import MetricFrame
from sklearn import metrics as skm
import seaborn as sns
import os.path as path
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, Polygon
import os, csv, shapely, shapely.wkt, descartes, geopy, pickle, cloudpickle

pd.set_option('display.max_rows', 10)
sns.set()
sns.color_palette("ch:s=.25,rot=-.25")
np.set_printoptions(suppress=True)
# pd.options.display.float_format = '{:.2f}'.format


Bad key "text.kerning_factor" on line 4 in
/Users/yhannnn/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
NUM_STATIONS_INC = 2

In [3]:
# NEED MODIFY
EPOCH = 250
LAMBDA = 0.01
FOLDER = 'fairst_all_models/e_250_lambda_0_01'
PRED_FILE_NAME = "inference_stations_2_fused_model_pred_0.01_1641302821.226266.csv"

In [4]:
CITY = "N_"
save_dir = CITY + 'reg_res/'
save_dir_preprocess = os.path.join(save_dir, CITY + 'preprocess_fig/')
save_dir_models = os.path.join(save_dir, CITY + 'model_res/') 
save_dir_forecast = os.path.join(save_dir, CITY + 'forecast/')
save_dir_fairst = os.path.join(save_dir, CITY + 'fairst/')

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
if not os.path.exists(save_dir_preprocess):
    os.makedirs(save_dir_preprocess)

if not os.path.exists(save_dir_models):
    os.makedirs(save_dir_models)
    
if not os.path.exists(save_dir_forecast):
    os.makedirs(save_dir_forecast)
    
if not os.path.exists(save_dir_fairst):
    os.makedirs(save_dir_fairst)

In [5]:
pred_df = pd.read_csv(os.path.join(save_dir_fairst, FOLDER, PRED_FILE_NAME), index_col = 0)
gt_df = pd.read_csv(os.path.join(save_dir_fairst, "test_df_all.csv"), index_col = 0)
pos_info = pd.read_csv(os.path.join(save_dir_fairst, "df_demo.csv"), index_col = 0)
income_info = pd.read_csv(os.path.join(save_dir_fairst, "df_income.csv"), index_col = 0)
income_info = income_info.groupby(by = "cell_num").mean().reset_index()
income_info.income_group = income_info.income_group.astype(int)
demo_raw = pd.read_csv(os.path.join(save_dir_fairst, "demo_raw_all.csv"))
pred_df.index = pd.to_datetime(pred_df.index)
gt_df.index = pd.to_datetime(gt_df.index)
gt_df = gt_df[gt_df.index.isin(pred_df.index)]
normalized_pop_df = pd.read_csv(os.path.join(save_dir_fairst, "normalized_pop_df.csv"), index_col = 0)

In [6]:
def df_transform(df):
    pos_l = df.columns
    df_trans = pd.DataFrame(columns = ["pos", "outflow"])

    for i in np.arange(len(pos_l)):
        pos = pos_l[i]
        temp = df[pos].to_frame()
        temp["pos"] = [temp.columns.values[0]]*len(temp)
        temp.rename(columns = {pos: "outflow"}, inplace = True)
        df_trans = df_trans.append(temp)
        
    return df_trans

In [7]:
pred_df_month = pred_df.groupby(pred_df.index.month).sum()

In [8]:
pred_df_month

,1_1,1_2,2_0,2_1,2_2,2_3,3_0,3_1,3_2,3_3,...,120_51,120_52,120_53,120_54,120_55,120_56,121_52,121_53,121_54,121_55
2,0.0,0.0,145.045424,0.0,0.0,0.0,145.045424,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.401944,1.401944,1.401944,1.401944
3,0.0,0.0,149.513051,0.0,0.0,0.0,149.513051,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,120.147551,0.0,0.0,0.0,120.147551,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
5,0.0,0.0,131.012124,0.0,0.0,0.0,131.012124,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
6,0.0,0.0,100.141354,0.0,0.0,0.0,100.141354,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,0.0,0.0,104.473343,0.0,0.0,0.0,104.473343,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,101.083362,0.0,0.0,0.0,101.083362,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
10,0.0,0.0,106.876845,0.0,0.0,0.0,106.876845,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
11,0.0,0.0,129.131638,0.0,0.0,0.0,129.131638,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000


In [9]:
df_transform_m = df_transform(pred_df_month).groupby('pos').sum()
df_transform_m = df_transform_m.reset_index()

In [10]:
# df_pred_trans = df_transform(pred_df_month)
pred_final = df_transform_m.merge(pos_info)

pred_final_m = pred_final.merge(income_info, on = 'cell_num')

In [11]:
pred_final_m.to_csv(os.path.join(save_dir_fairst, "fairst_forecast_inc_{}.csv".format(NUM_STATIONS_INC)))